# Entendiendo el proceso

In [2]:
import torch as tr
import torch.nn as nn

In [3]:
# Entendiendo unsqueeze() con ejemplos

# unsqueeze() agrega una dimensión de tamaño 1 en la posición especificada
tensor_1d = tr.tensor([1, 2, 3])
print("Tensor original 1D:")
print(tensor_1d)
print("Shape:", tensor_1d.shape)

# Agregar dimensión en posición 0
unsqueezed_0 = tensor_1d.unsqueeze(0)
print("\nunsqueeze(0) - agregar dimensión al inicio:")
print(unsqueezed_0)
print("Shape:", unsqueezed_0.shape)

# Agregar dimensión en posición 1
unsqueezed_1 = tensor_1d.unsqueeze(1)
print("\nunsqueeze(1) - agregar dimensión al final:")
print(unsqueezed_1)
print("Shape:", unsqueezed_1.shape)

# Ejemplo con tensor 2D
tensor_2d = tr.tensor([[1, 2], [3, 4]])
print("\n\nTensor original 2D:")
print(tensor_2d)
print("Shape:", tensor_2d.shape)

# Agregar dimensión en diferentes posiciones
print("\nunsqueeze(0) en 2D - nueva dimensión al inicio:")
print(tensor_2d.unsqueeze(0))
print("Shape:", tensor_2d.unsqueeze(0).shape)

print("\nunsqueeze(1) en 2D - nueva dimensión en el medio:")
print(tensor_2d.unsqueeze(1))
print("Shape:", tensor_2d.unsqueeze(1).shape)

print("\nunsqueeze(2) en 2D - nueva dimensión al final:")
print(tensor_2d.unsqueeze(2))
print("Shape:", tensor_2d.unsqueeze(2).shape)

# También se puede usar con índices negativos
print("\nunsqueeze(-1) - última posición:")
print(tensor_1d.unsqueeze(-1))
print("Shape:", tensor_1d.unsqueeze(-1).shape)

Tensor original 1D:
tensor([1, 2, 3])
Shape: torch.Size([3])

unsqueeze(0) - agregar dimensión al inicio:
tensor([[1, 2, 3]])
Shape: torch.Size([1, 3])

unsqueeze(1) - agregar dimensión al final:
tensor([[1],
        [2],
        [3]])
Shape: torch.Size([3, 1])


Tensor original 2D:
tensor([[1, 2],
        [3, 4]])
Shape: torch.Size([2, 2])

unsqueeze(0) en 2D - nueva dimensión al inicio:
tensor([[[1, 2],
         [3, 4]]])
Shape: torch.Size([1, 2, 2])

unsqueeze(1) en 2D - nueva dimensión en el medio:
tensor([[[1, 2]],

        [[3, 4]]])
Shape: torch.Size([2, 1, 2])

unsqueeze(2) en 2D - nueva dimensión al final:
tensor([[[1],
         [2]],

        [[3],
         [4]]])
Shape: torch.Size([2, 2, 1])

unsqueeze(-1) - última posición:
tensor([[1],
        [2],
        [3]])
Shape: torch.Size([3, 1])


# Analizamos la función original

In [3]:
class WeightedFamiliesLayer(nn.Module):
    """This is the original weighted family layer, where each model has a weight per PF."""
    def __init__(self, num_models, num_classes):
        super(WeightedFamiliesLayer, self).__init__()
        self.weights = nn.Parameter(tr.rand(num_models, num_classes))

    def forward(self, x): # x: (num_models, batch, num_classes)
        # Weights need to be broadcasted over the batch dimension
        return tr.sum(x * self.weights.view(self.weights.shape[0], 1, -1), dim=0)

In [6]:
# Entendiendo WeightedFamiliesLayer paso a paso con valores determinísticos

import torch as tr

# Definimos pesos fijos (num_models=3, num_classes=2)
weights = tr.tensor([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]], dtype=tr.float32)  # (3, 2)
print("Pesos (weights):")
print(weights)
print("Shape:", weights.shape)

# Input fijo: (num_models=3, batch=2, num_classes=2)
x = tr.tensor([[[1.0, 2.0], [3.0, 4.0]], 
                [[5.0, 6.0], [7.0, 8.0]], 
                [[9.0, 10.0], [11.0, 12.0]]], dtype=tr.float32)
print("\nInput x:")
print(x)
print("Shape:", x.shape)

# Paso 1: View de weights para broadcasting
weights_view = weights.view(weights.shape[0], 1, -1)
print("\nPaso 1: weights.view(weights.shape[0], 1, -1)")
print("weights_view shape:", weights_view.shape)
print("weights_view:")
print(weights_view)

# Paso 2: Multiplicación con broadcasting
print("\nPaso 2: x * weights_view (broadcasting)")
multiplied = x * weights_view
print("multiplied shape:", multiplied.shape)
print("multiplied:")
print(multiplied)

# Paso 3: Suma a lo largo de dim=0 (num_models)
print("\nPaso 3: tr.sum(multiplied, dim=0)")
summed = tr.sum(multiplied, dim=0)
print("summed shape:", summed.shape)
print("summed (output final):")
print(summed)

# Verificación manual para un elemento
print("\nVerificación manual:")
print("Para batch=0, class=0: 1*1 + 5*3 + 9*5 =", 1*1 + 5*3 + 9*5)
print("Para batch=0, class=1: 2*2 + 6*4 + 10*6 =", 2*2 + 6*4 + 10*6)
print("Etc.")

Pesos (weights):
tensor([[1., 2.],
        [3., 4.],
        [5., 6.]])
Shape: torch.Size([3, 2])

Input x:
tensor([[[ 1.,  2.],
         [ 3.,  4.]],

        [[ 5.,  6.],
         [ 7.,  8.]],

        [[ 9., 10.],
         [11., 12.]]])
Shape: torch.Size([3, 2, 2])

Paso 1: weights.view(weights.shape[0], 1, -1)
weights_view shape: torch.Size([3, 1, 2])
weights_view:
tensor([[[1., 2.]],

        [[3., 4.]],

        [[5., 6.]]])

Paso 2: x * weights_view (broadcasting)
multiplied shape: torch.Size([3, 2, 2])
multiplied:
tensor([[[ 1.,  4.],
         [ 3.,  8.]],

        [[15., 24.],
         [21., 32.]],

        [[45., 60.],
         [55., 72.]]])

Paso 3: tr.sum(multiplied, dim=0)
summed shape: torch.Size([2, 2])
summed (output final):
tensor([[ 61.,  88.],
        [ 79., 112.]])

Verificación manual:
Para batch=0, class=0: 1*1 + 5*3 + 9*5 = 61
Para batch=0, class=1: 2*2 + 6*4 + 10*6 = 88
Etc.


## Vemos como funciona la capa `Linear` de torch

In [ ]:
layer = nn.Linear(in_features=3, out_features=2)

input = tr.tensor([[0.5, 0.6, 0.7], [0.8, 0.9, 1.0]], dtype=tr.float32) 
print("Input size:")
print(input.size())

output = layer(input)
print("Output size:")
print(output.size())

Input size:
torch.Size([2, 2, 3])
Output size:
torch.Size([2, 2, 2])


In [12]:
# Mostramos los parámetros de la capa lineal
print("Pesos (weight):", layer.weight.shape)
print("Bias:", layer.bias.shape)
print("\nValores de pesos:")
print(layer.weight)
print("\nValores de bias:")
print(layer.bias)

Pesos (weight): torch.Size([2, 3])
Bias: torch.Size([2])

Valores de pesos:
Parameter containing:
tensor([[-0.4124, -0.2507, -0.2004],
        [ 0.0256,  0.0472, -0.1147]], requires_grad=True)

Valores de bias:
Parameter containing:
tensor([-0.4679,  0.3517], requires_grad=True)


Seteamos manualmente el valor de los parametros:

In [10]:
# Crear nuevos valores (ejemplo simple)
new_weight = tr.tensor([[0.5, 0.6, 0.7], [0.8, 0.9, 1.0]], dtype=tr.float32)  # (2, 3)
new_bias = tr.tensor([0.1, 0.2], dtype=tr.float32)  # (2,)

# Asignar usando .data para evitar problemas con gradientes
layer.weight.data = new_weight
layer.bias.data = new_bias

print("\nNuevos valores de weight:")
print(layer.weight)
print("\nNuevos valores de bias:")
print(layer.bias)


Nuevos valores de weight:
Parameter containing:
tensor([[0.5000, 0.6000, 0.7000],
        [0.8000, 0.9000, 1.0000]], requires_grad=True)

Nuevos valores de bias:
Parameter containing:
tensor([0.1000, 0.2000], requires_grad=True)


In [11]:
# Operación matemática manual
# La capa lineal hace: output = input @ weight.T + bias
# Pero PyTorch maneja automáticamente las dimensiones

# Para entender: input tiene forma (batch, seq, in_features)
# weight tiene forma (out_features, in_features)
# bias tiene forma (out_features,)

# La operación es: output[b, s, :] = input[b, s, :] @ weight.T + bias

# Vamos a calcular manualmente para el primer ejemplo del batch
primer_input = input[0, 0, :]  # (10,)
print("Primer input (10 features):", primer_input)

# Multiplicación matricial: primer_input @ weight.T
manual_output = primer_input @ m.weight.T + m.bias
print("Output manual para primer input:", manual_output)

# Comparar con el output de PyTorch
pytorch_output = output[0, 0, :]
print("Output de PyTorch para primer input:", pytorch_output)

print("¿Son iguales?", torch.allclose(manual_output, pytorch_output))

Primer input (10 features): tensor([ 0.1888, -1.1459,  0.9217])
Output manual para primer input: tensor([0.4529, 0.5740], grad_fn=<AddBackward0>)
Output de PyTorch para primer input: tensor([0.4529, 0.5740], grad_fn=<SliceBackward0>)
¿Son iguales? True


In [12]:
# Cantidad de parámetros
total_params = sum(p.numel() for p in m.parameters())
print(f"Total de parámetros: {total_params}")
print(f"Parámetros en weight: {m.weight.numel()}")
print(f"Parámetros en bias: {m.bias.numel()}")

# Para in_features=10, out_features=5:
# weight: 10 * 5 = 50 parámetros
# bias: 5 parámetros
# Total: 55 parámetros

Total de parámetros: 8
Parámetros en weight: 6
Parámetros en bias: 2


In [13]:
# Setear valores manuales a los pesos y bias
print("Valores originales de weight:")
print(m.weight)
print("\nValores originales de bias:")
print(m.bias)

# Crear nuevos valores (ejemplo simple)
new_weight = torch.tensor([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]], dtype=torch.float32)  # (2, 3)
new_bias = torch.tensor([0.1, 0.2], dtype=torch.float32)  # (2,)

# Asignar usando .data para evitar problemas con gradientes
m.weight.data = new_weight
m.bias.data = new_bias

print("\nNuevos valores de weight:")
print(m.weight)
print("\nNuevos valores de bias:")
print(m.bias)

# Probar con el mismo input
new_output = m(input)
print("\nNuevo output con pesos seteados:")
print(new_output)

# Explicación: Ahora weight[0] = [1,0,0], así que output[0] = input[0]*1 + input[1]*0 + input[2]*0 + bias[0]
# Similar para output[1]

Valores originales de weight:
Parameter containing:
tensor([[-0.1526,  0.3052,  0.4716],
        [ 0.2453, -0.0215,  0.1254]], requires_grad=True)

Valores originales de bias:
Parameter containing:
tensor([0.3967, 0.3876], requires_grad=True)

Nuevos valores de weight:
Parameter containing:
tensor([[1., 0., 0.],
        [0., 1., 0.]], requires_grad=True)

Nuevos valores de bias:
Parameter containing:
tensor([0.1000, 0.2000], requires_grad=True)

Nuevo output con pesos seteados:
tensor([[[ 0.2888, -0.9459],
         [-0.9840,  0.8339]]], grad_fn=<ViewBackward0>)
